In [ ]:
from pathlib import Path
from lib.noise_sampler import generate_noise_graphs

RULES_PATH = Path("lib/sampling_rules.json")
noise_graphs = generate_noise_graphs(n=200, max_events=24, rules_json_path=RULES_PATH, seed=42)

In [1]:
from lib.target_sampler import generate_target_graphs

target_graphs = generate_target_graphs()

In [2]:
from lib.WL2vec import graph_to_fingerprint

# noise_fingerprints = {str(i): graph_to_fingerprint(g) for i, g in enumerate(noise_graphs)}
target_fingerprints = {str(i): graph_to_fingerprint(g) for i, g in enumerate(target_graphs)}

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.6, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,

In [ ]:
from lib.pinecone import upsert

upsert(fingerprints=noise_fingerprints)
upsert(fingerprints=target_fingerprints, metadata={ 'target': 'yes' })

In [ ]:

from lib.pinecone import query

target_base_graph = str(len(noise_fingerprints))
results = query(target_fingerprints[target_base_graph], 10)

matches = results.matches


In [ ]:
all_graphs = noise_graphs + target_graphs

for match in results.matches:
    print(match.id)
    print("-----------")
    all_graphs[int(match.id)].visualize()